# Region Map

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import urllib.parse

# Import API keys- make sure you add your keys to the api_config file
from api_config import g_key
from api_config import api_key

In [ ]:
# map_sample_pd = clean_data_pd.sample(1000)
# map_sample_pd

In [ ]:
# ## If closing this Jupyter Notebook, write map_sample_pd to CSV instead of re-creating the sample DataFrame
# map_sample_pd.to_csv('Resources/map_sample1000.csv', encoding='utf-8')

In [2]:
## If opening this Jupyter Notebook, read in the CSV "map_sample1000.csv" and create a map_sample_pd DataFrame
map_sample_pd = pd.read_csv('Resources/map_sample1000.csv', encoding='utf-8')
del map_sample_pd['Unnamed: 0']
del map_sample_pd['Unnamed: 0.1']
map_sample_pd

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
0,France,"With its 50% Sémillon, this ripe wine has the ...",Cuvée des Conti,88,16.0,Southwest France,Bergerac Sec,NaN,Château Tour des Gendres 2014 Cuvée des Conti ...,Bordeaux-style White Blend,Château Tour des Gendres
1,US,"Concentrated, bright and focused fruit flavors...",Perli Vineyards,91,31.0,California,Mendocino Ridge,NaN,Edmeades 2013 Perli Vineyards Zinfandel (Mendo...,Zinfandel,Edmeades
2,US,This 60% Merlot and 40% Cabernet Sauvignon ble...,SageCliffe 100 Red,84,22.0,Washington,Columbia Valley (WA),Columbia Valley,Cave B 2005 SageCliffe 100 Red Red (Columbia V...,Red Blend,Cave B
3,US,"One of the better Sangioveses out there, altho...",Estate,87,28.0,California,Amador County,Sierra Foothills,IL Gioiello 2006 Estate Sangiovese (Amador Cou...,Sangiovese,IL Gioiello
4,Australia,This reasonably full-bodied Chardonnay manages...,Tarraford Vineyard,90,42.0,Victoria,Yarra Valley,NaN,Giant Steps 2013 Tarraford Vineyard Chardonnay...,Chardonnay,Giant Steps
...,...,...,...,...,...,...,...,...,...,...,...
995,US,Made in even smaller amounts than the vintage ...,The Red Hat Old Vine Field Blend,93,37.0,California,Napa Valley,Napa,Sheldon 2015 The Red Hat Old Vine Field Blend ...,Red Blend,Sheldon
996,Italy,"Aromas of toast, scorched earth, black plum, e...",Conca Tre Pile,90,35.0,Piedmont,Barbera d'Alba,NaN,Poderi Aldo Conterno 2012 Conca Tre Pile (Bar...,Barbera,Poderi Aldo Conterno
997,US,"Exaggerated, burnt aromas of sun-dried tomato,...",Picnic Hill Vineyard Late Harvest,82,14.0,California,Amador County,Sierra Foothills,Audubon Cellars 1997 Picnic Hill Vineyard Late...,Zinfandel,Audubon Cellars
998,Argentina,"Aromas of banana, tropical fruits and spun sug...",Phebus Un-Oaked Estate Bottled,84,12.0,Mendoza Province,Mendoza,NaN,Fabre Montmayou 2013 Phebus Un-Oaked Estate Bo...,Chardonnay,Fabre Montmayou


In [16]:
# Test API parameters for locations search.
query = f'{map_sample_pd["winery"][0]},{map_sample_pd["region_1"][0]},{map_sample_pd["province"][0]}'
place = urllib.parse.quote(query)

# Base URL 
base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}&key={api_key}')

# Run a request to view json structure
response = requests.get(base_url).json()
pprint.pprint(response)

{'documentation': 'https://opencagedata.com/api',
 'licenses': [{'name': 'see attribution guide',
               'url': 'https://opencagedata.com/credits'}],
 'rate': {'limit': 2500, 'remaining': 1479, 'reset': 1602028800},
 'results': [{'annotations': {'DMS': {'lat': "46° 0' 0.00000'' N",
                                      'lng': "2° 0' 0.00000'' E"},
                              'MGRS': '31TDL2256794533',
                              'Maidenhead': 'JN16aa00aa',
                              'Mercator': {'x': 222638.982, 'y': 5749599.546},
                              'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=16/46.00000/2.00000&layers=N',
                                      'url': 'https://www.openstreetmap.org/?mlat=46.00000&mlon=2.00000#map=16/46.00000/2.00000'},
                              'UN_M49': {'regions': {'EUROPE': '150',
                                                     'FR': '250',
                                                     'WE

In [4]:
# Hit the OpenCageData API for each winery.
# Set up a Dictionary to hold reponse info for:
town=[]
lat=[]
lng=[]


print("Beginning search for nearby wineries... \n-----------------------------")

# Loop through the DataFrame to append a lat, long, and town for each winery
for index, row in map_sample_pd.iterrows():
   
    # Query OpenCage API with the Winery Name, Region 1, Province, and Country
    query = f'{row["winery"]},{row["region_1"]},{row["province"]},{row["country"]}'
    place = urllib.parse.quote(query)

    base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}&key={api_key}')

    try:
        print(f"Searching for winery: {query}")
        response = requests.get(base_url).json()
        
        # Selecting responses with a medium to high confidence score: >4 or within 10 km of query result
        num_responses = len(response['results'])
        print(f'num of responses {num_responses}')
        for x in range(num_responses):
            print(f'response{x}')
            confidence = response['results'][x]['confidence'] 
            if confidence >4:
                print(f'confidence:{confidence}')
                town.append(response['results'][x]['components']['town'])
                lat.append(response['results'][x]['geometry']['lat'])
                lng.append(response['results'][x]['geometry']['lng'])
                print(f'---> {query} appended')
                print(" ")
                break
            elif x == (num_responses - 1):
                print(f'Not confident: {confidence} - Skipping {query}...')
                print(" ")
                town.append(None)
                lat.append(None)
                lng.append(None)
            else:
                print(f'Not confident: {confidence} - Next result...')
                print(" ")
    except:
        print(f"X--X No wineries found. Skipping {query}...")
        print(" ")
        town.append(None)
        lat.append(None)
        lng.append(None)

Beginning search for nearby wineries... 
-----------------------------
Searching for winery: Château Tour des Gendres,Bergerac Sec,Southwest France,France
num of responses 4
response0
Not confident: 1 - Next result...
 
response1
Not confident: 2 - Next result...
 
response2
Not confident: 2 - Next result...
 
response3
confidence:10
X--X No wineries found. Skipping Château Tour des Gendres,Bergerac Sec,Southwest France,France...
 
Searching for winery: Edmeades,Mendocino Ridge,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:7
---> Edmeades,Mendocino Ridge,California,US appended
 
Searching for winery: Cave B,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Cave B,Columbia Valley (WA),Washington,US...
 
Searching for winery: IL Gioiello,Amador County,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
r

num of responses 2
response0
confidence:5
---> Gnarly Head,California,California,US appended
 
Searching for winery: Columbia Winery,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Columbia Winery,Columbia Valley (WA),Washington,US...
 
Searching for winery: Fox Run,Seneca Lake,New York,US
num of responses 2
response0
confidence:5
---> Fox Run,Seneca Lake,New York,US appended
 
Searching for winery: Mount Palomar,Temecula,California,US
num of responses 2
response0
confidence:5
---> Mount Palomar,Temecula,California,US appended
 
Searching for winery: Gnarly Head,California,California,US
num of responses 2
response0
confidence:5
---> Gnarly Head,California,California,US appended
 
Searching for winery: Thirsty Owl Wine Company,Finger Lakes,New York,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Thirsty Ow

num of responses 10
response0
confidence:8
---> Chanson Père et Fils,Chablis,Burgundy,France appended
 
Searching for winery: Savage Grace,Rattlesnake Hills,Washington,US
num of responses 1
response0
confidence:9
X--X No wineries found. Skipping Savage Grace,Rattlesnake Hills,Washington,US...
 
Searching for winery: Louis Latour,Meursault,Burgundy,France
num of responses 10
response0
confidence:7
---> Louis Latour,Meursault,Burgundy,France appended
 
Searching for winery: Domaine Houchart,Côtes de Provence,Provence,France
num of responses 1
response0
confidence:9
X--X No wineries found. Skipping Domaine Houchart,Côtes de Provence,Provence,France...
 
Searching for winery: Tablas Creek,Paso Robles,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:6
---> Tablas Creek,Paso Robles,California,US appended
 
Searching for winery: Paul Jaboulet Aîné,Hermitage,Rhône Valley,France
num of responses 4
response0
Not confident: 1 - Next result...
 
r

num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping T'Jara,North Fork of Long Island,New York,US...
 
Searching for winery: A.R. Lenoble ,Champagne,Champagne,France
num of responses 7
response0
confidence:7
---> A.R. Lenoble ,Champagne,Champagne,France appended
 
Searching for winery: Koenig,Alsace,Alsace,France
num of responses 7
response0
confidence:8
X--X No wineries found. Skipping Koenig,Alsace,Alsace,France...
 
Searching for winery: Dumas Station,Walla Walla Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Dumas Station,Walla Walla Valley (WA),Washington,US...
 
Searching for winery: Valentin Bianchi,Mendoza,Mendoza Province,Argentina
num of responses 1
response0
Not confident: 1 - Skipping Valentin Bianchi,Mendoza,Mendoza Province,Argentina...
 
Searching for winery: Pago de los Capellanes,Ribera del Duero,Norther

num of responses 2
response0
confidence:6
---> WillowBrook Cellars,Sonoma Mountain,California,US appended
 
Searching for winery: Rattalino,Barbaresco,Piedmont,Italy
num of responses 1
response0
confidence:8
---> Rattalino,Barbaresco,Piedmont,Italy appended
 
Searching for winery: La Chertosa,Sonoma Valley,California,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping La Chertosa,Sonoma Valley,California,US...
 
Searching for winery: Cebada,Santa Ynez Valley,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Cebada,Santa Ynez Valley,California,US...
 
Searching for winery: Atticus,Willamette Valley,Oregon,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Atticus,Willamette Valley,Oregon,US...
 
Searching for winery: Dominio del Plata,Mendoza,Mendoza Province,A

...

num of responses 1
response0
Not confident: 1 - Skipping Emilio Moro,Ribera del Duero,Northern Spain,Spain...
 
Searching for winery: Watts,California,California,US
num of responses 2
response0
confidence:7
X--X No wineries found. Skipping Watts,California,California,US...
 
Searching for winery: Kiona,Yakima Valley,Washington,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Kiona,Yakima Valley,Washington,US...
 
Searching for winery: Lockwood,Monterey County,California,US
num of responses 2
response0
confidence:7
X--X No wineries found. Skipping Lockwood,Monterey County,California,US...
 
Searching for winery: Col d'Orcia,Sant'Antimo,Tuscany,Italy
num of responses 1
response0
confidence:9
---> Col d'Orcia,Sant'Antimo,Tuscany,Italy appended
 
Searching for winery: Tommasi,Delle Venezie,Northeastern Italy,Italy
num of responses 1
response0
Not confident: 1 - Skipping Tommasi,Delle Venezie,Northeastern Italy,Italy

num of responses 4
response0
confidence:9
X--X No wineries found. Skipping Domaine Faiveley,Echézeaux,Burgundy,France...
 
Searching for winery: Morell-Peña,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Morell-Peña,Columbia Valley (WA),Washington,US...
 
Searching for winery: Castello di Querceto,Colli della Toscana Centrale,Tuscany,Italy
num of responses 1
response0
confidence:9
---> Castello di Querceto,Colli della Toscana Centrale,Tuscany,Italy appended
 
Searching for winery: Jean-Marc Bernhard,Alsace,Alsace,France
num of responses 3
response0
confidence:9
X--X No wineries found. Skipping Jean-Marc Bernhard,Alsace,Alsace,France...
 
Searching for winery: Lynmar,Russian River Valley,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping Lynmar,Russian River Valley,California,US...
 
Searching for

num of responses 2
response0
confidence:6
---> Rancho Sisquoc,Santa Barbara County,California,US appended
 
Searching for winery: Trentadue,Alexander Valley,California,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:7
X--X No wineries found. Skipping Trentadue,Alexander Valley,California,US...
 
Searching for winery: Vivác Winery,New Mexico,New Mexico,US
num of responses 1
response0
Not confident: 1 - Skipping Vivác Winery,New Mexico,New Mexico,US...
 
Searching for winery: Bianchi,Santa Barbara County,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
Not confident: 1 - Skipping Bianchi,Santa Barbara County,California,US...
 
Searching for winery: Paltrinieri,Lambrusco di Sorbara,Central Italy,Italy
num of responses 1
response0
Not confident: 1 - Skipping Paltrinieri,Lambrusco di Sorbara,Central Italy,Italy...
 
Searching for winery: Rivera,Castel del Monte,Southern Italy,Italy
num of responses 1
response0


num of responses 2
response0
confidence:5
---> Iron Horse,Sonoma County,California,US appended
 
Searching for winery: Bodegas Peñafiel,Ribera del Duero,Northern Spain,Spain
num of responses 1
response0
Not confident: 1 - Skipping Bodegas Peñafiel,Ribera del Duero,Northern Spain,Spain...
 
Searching for winery: Bruliam,Sonoma County,California,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
Not confident: 1 - Skipping Bruliam,Sonoma County,California,US...
 
Searching for winery: Virage,Napa Valley,California,US
num of responses 2
response0
confidence:5
---> Virage,Napa Valley,California,US appended
 
Searching for winery: Trio Vintners,Walla Walla Valley (WA),Washington,US
num of responses 2
response0
confidence:5
---> Trio Vintners,Walla Walla Valley (WA),Washington,US appended
 
Searching for winery: Hess Collection,Lake County,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
Not confident: 1 - Skipping Hess Colle

num of responses 1
response0
confidence:9
---> Cantine dei Colli Tramonte,Veneto,Veneto,Italy appended
 
Searching for winery: Jean-Marc Bernhard,Crémant d'Alsace,Alsace,France
num of responses 2
response0
Not confident: 1 - Next result...
 
response1
Not confident: 3 - Skipping Jean-Marc Bernhard,Crémant d'Alsace,Alsace,France...
 
Searching for winery: Bibbiano,Chianti Classico,Tuscany,Italy
num of responses 1
response0
Not confident: 1 - Skipping Bibbiano,Chianti Classico,Tuscany,Italy...
 
Searching for winery: Nottingham Cellars,Livermore Valley,California,US
num of responses 2
response0
confidence:6
---> Nottingham Cellars,Livermore Valley,California,US appended
 
Searching for winery: Novy,North Coast,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
Not confident: 1 - Skipping Novy,North Coast,California,US...
 
Searching for winery: Cadaretta,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...

In [6]:
# Count the length of the appended lists to ensure they are the same as the existing DataFrame- otherwise an Error occurs 
town, lat, lng
len(town)

1000

In [7]:
# Add the results into the existing DataFrame. Then Dropna for lat/lng and save as wine_map_pd DataFrame
map_sample_pd['town'] = town
map_sample_pd['lat'] = lat
map_sample_pd['lng'] = lng
wine_map_pd = map_sample_pd.dropna(subset=['lat', 'lng'])
wine_map_pd

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,town,lat,lng
1,US,"Concentrated, bright and focused fruit flavors...",Perli Vineyards,91,31.0,California,Mendocino Ridge,NaN,Edmeades 2013 Perli Vineyards Zinfandel (Mendo...,Zinfandel,Edmeades,Ridge,39.336550,-123.301120
4,Australia,This reasonably full-bodied Chardonnay manages...,Tarraford Vineyard,90,42.0,Victoria,Yarra Valley,NaN,Giant Steps 2013 Tarraford Vineyard Chardonnay...,Chardonnay,Giant Steps,Healesville,-37.651340,145.522713
7,US,"Notes of sweet pea blossoms, freesia and ripe ...",NaN,88,15.0,New York,Finger Lakes,Finger Lakes,Goose Watch 2011 Gewürztraminer-Riesling (Fing...,Gewürztraminer-Riesling,Goose Watch,Stanley,42.856417,-77.125804
9,US,"An enormously complex wine, but one that's no...",J. Schram Rosé,92,140.0,California,Napa-Mendocino-Sonoma-Marin,North Coast,Schramsberg 2005 J. Schram Rosé Sparkling (Nap...,Sparkling Blend,Schramsberg,California Hot Springs,35.881776,-118.656086
16,US,"Plump aromas of white peach, lemon syrup, butt...",Rincon,92,44.0,California,Arroyo Grande Valley,Central Coast,Talley 2015 Rincon Chardonnay (Arroyo Grande V...,Chardonnay,Talley,Arroyo Grande,35.112711,-120.580770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,US,"While creamy and voluminous, this full-bodied ...",Barrel Fermented,88,18.0,New York,Finger Lakes,Finger Lakes,Damiani 2013 Barrel Fermented Chardonnay (Fing...,Chardonnay,Damiani,New York,40.815120,-73.958700
988,US,This wine offers high-toned cherry amidst a fr...,Two Range,91,25.0,California,Napa Valley,Napa,Rutherford Ranch 2015 Two Range Red (Napa Valley),Red Blend,Rutherford Ranch,California City,35.113893,-117.982087
989,France,"At first taste, this wine seems more new world...",NaN,94,80.0,Bordeaux,Margaux,NaN,Château Rauzan-Ségla 2000 Margaux,Bordeaux-style Red Blend,Château Rauzan-Ségla,Bordeaux,44.840440,-0.580500
994,US,Has indistinct watery aromas of lemons and but...,NaN,81,9.0,Washington,Yakima Valley,Columbia Valley,Hoodsport 1998 Sémillon (Yakima Valley),Sémillon,Hoodsport,Yakima,46.547726,-120.476597


In [8]:
# Count the length of the new DataFrame to determine how many results remain for mapping 
len(wine_map_pd)

306

In [9]:
# ## If closing Jupyter Notebook, write the API output to CSV ".csv" instead of re-reunning OpenCageData API calls
# wine_map_pd.to_csv('Resources/wine_map.csv', encoding='utf-8')

In [ ]:
# ## If opening Jupyter Notebook, read in the saved CSV ".csv" instead of re-reunning OpenCageData API calls
# wine_map_pd = pd.read_csv('Resources/wine_map.csv', encoding='utf-8')

In [10]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [20]:
# Store 'lat' and 'lng' into  locations 
locations = wine_map_pd[["lat", "lng"]].astype(float)
rating = wine_map_pd["points"].astype(float)
price = wine_map_pd["price"].astype(float)

# Create Humidity Heatmap layer
fig = gmaps.figure()

heat_rating = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=rating.max())
heat_price = gmaps.heatmap_layer(locations, weights=price, 
                                 dissipating=True, max_intensity=price.max(), opacity=0.3, gradient=['blue','red'])

fig.add_layer(heat_rating)
fig.add_layer(heat_price)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{winery}</dd>
<dt>Vintage</dt><dd>{title}</dd>
<dt>Rating</dt><dd>{points}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Province</dt><dd>{province}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
winery_info = [info_box_template.format(**row) for index, row in wine_map_pd.iterrows()]
locations = wine_map_pd[["lat", "lng"]] 


In [21]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=winery_info, display_info_box=None)

fig.add_layer(heat_rating)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))